## Lecture 8
We will first go through how to generate plots using Matplotlib, a plotting package for Python.

We will for the most of time use it to generate 2D plots. In the rare case of multiple parameters and if you are interested, 3D plots may also be helpful.

Key points for using matplotlib:
1. 2D line and curves
2. Adjusting colors, thickness, dash/dot and other plotting styles
3. Adjusting plot ranges
4. Making multiple plots on the same graph; adding plot legend
5. Save graphs

### `Matplotlib`

Usually we don't need to use the entire `matplotlib` package. So we do


```python
import matplotlib.pyplot as plt
```

In [ ]:
### Import the plotting function
import matplotlib.pyplot as plt

I like to modify the plot properties to make the figure looks nicer.  See below for the default settings I normally use.  Feel free to play with the settings however you like.  You can also compare the difference in plots with or without the following configurations.

In [ ]:
from matplotlib import rc

plt.rcParams['xtick.labelsize']=25      # change the tick label size for x axis
plt.rcParams['ytick.labelsize']=25      # change the tick label size for x axis
plt.rcParams['axes.linewidth']=3        # change the line width of the axis
plt.rcParams['xtick.major.width'] = 3   # change the tick line width of x axis
plt.rcParams['ytick.major.width'] = 3   # change the tick line width of y axis
rc('text', usetex=False)                # disable LaTeX rendering in plots
rc('font',**{'family':'DejaVu Sans'})   # set the font of the plot to be DejaVu Sans

Now let's do a simple example.  Make a plot for $y=\sin(x)$ for $x=[0,8]$.

In [ ]:
import numpy as np

x = np.linspace(0, 8, 200)    # we first generate an array of x values for computing sin(x)
plt.plot(x, np.sin(x))

Let's now make this plot looks nicer

In [ ]:
# We can set the figure size by first creating a figure object
# Here I set the figure to be 9x6in with a dots per inch (dpi) to be 80
fig = plt.figure(num = 1, figsize = (9, 6), dpi = 80, facecolor = None, edgecolor = 'k')

# I made the plotting line to have width 3, and plot in black
plt.plot(x, np.sin(x), linewidth = 3, color = 'k')
plt.xlim([-0.1, 8.1])   #setting x-axis range
plt.ylim([-1.1, 1.1])   #setting y-axis range

Now let's also do a scatter plot

In [ ]:
fig = plt.figure(num = 1, figsize = (9, 6), dpi = 80, facecolor = None, edgecolor = 'k')

x = np.linspace(0, 8, 50)    # we first generate an array of x values for computing sin(x)

plt.plot(x, np.sin(x), linewidth = 3, color = 'k', label = "line")
# I made a scatter plot to have marker size 200, and marker type square
plt.scatter(x, np.sin(x), s = 200, marker = 's', label = "sin scatter")
plt.xlim([-0.1, 8.1])   #setting x-axis range
plt.ylim([-1.1, 1.1])   #setting y-axis range
plt.legend(fontsize = 25) # you can also create legends for different lines
plt.savefig("sinx.pdf") # now you save the figure in the same folder as your jupyter notebook

For more information on how to make figures.  See this link: https://matplotlib.org/stable/tutorials/index.html

### Bisection Method

In [ ]:
def bisection(f, a, b, tol=1e-3):
  fa = f(a)
  fb = f(b)
  root = []
  if fa*fb > 0:
    raise ValueError("There is no root in interval [a,b], try again.")
  else:
    while (b-a)/2.0 > tol:
      c = (a+b)/2.0
      root.append(c)
      fc = f(c)
      if fc == 0:
        break
      else:
        if fa*fc < 0:
          b = c
          fb = fc
        else:
          a = c
          fa = fc
    return (a+b)/2.0, root

In [ ]:
def func(x):
  return x**2 - 4.0

final_root, all_root = bisection(func, 0, 3, tol=1e-6)

In [ ]:
fig = plt.figure(num = 1, figsize = (9, 6), dpi = 80, facecolor = None, edgecolor = 'k')

plt.scatter(range(len(all_root)), all_root, s = 100)
plt.axhline(y = 2.0, color = 'k', linewidth = 2)

In [ ]:
all_root

### Limit of Accuracy

In [ ]:
# function from Sauer P.46
# this function has a root at x = 2/3
def new_func(x):
  return x**3 - 2*x**2 + 4*x/3 - 8.0/27

for tol in [1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]:
  final_root, all_root = bisection(new_func, 0.0, 1.0, tol=tol)
  print("Estimated Root: ", final_root, " Number of Steps:", len(all_root), " Function Value at Estimated Root: ", new_func(final_root))

Let's try using `Scipy`'s Brent Method.  For more information on how to use root finding in `scipy`, check here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root_scalar.html#scipy.optimize.root_scalar

Other more specific methods for root finding: https://docs.scipy.org/doc/scipy/reference/optimize.html#root-finding



In [ ]:
from scipy import optimize

sol = optimize.root_scalar(new_func, bracket=[0, 1], method='brentq')
sol.root, sol.iterations, sol.function_calls

Why we can't get a more accurate solution?

Because of machine precision!

In [ ]:
x = np.linspace(2.0/3.0 - 1e-5, 2.0/3.0 + 1e-5, 100)

fig = plt.figure(num = 1, figsize = (9, 6), dpi = 80, facecolor = None, edgecolor = 'k')

plt.scatter(x, new_func(x), linewidth = 2)
plt.plot(x, new_func(x), linewidth = 2)
plt.axhline(y = 0, color = 'k', ls = '-.', linewidth = 2)
plt.axvline(x = 2.0/3.0, color = 'k', ls = '-.', linewidth = 2)

### Backward and Forward Error

Assume that $f$ is a function and that $r$ is a root, meaning that it satisfies $f(r) = 0$. Assume that $x_a$ is an approximation to $r$. For the root-finding problem, the **backward error** of the approximation $x_a$ is $|f(x_a)|$ and the **forward error** is $|r − x_a|$.

In the example function above:
$$ f(x) = x^3 - 2x^2 + \frac{4}{3}x - \frac{8}{27} $$,

the **backward error** $|f(x_a)|$ is around machine precision $\epsilon_\text{mach}\approx 2.2\times10^{-16}$, while the **forward error** $|r − x_a|$ is around $10^{-5}$.

Since we cannot decrease the **backward error** any more, we cannot further decrease the **forward error** either.

#### Example:

The function $f(x) = \sin x − x$ has a triple root at $r = 0$. Find the forward and backward error of the approximate root $x_c = 0.001$.

**Solution:**

We know that the root $r=0$, so the forward error by definition will be $|r-x_c| = |0-0.001| = 10^{-3}$.

The backward error will be $|f(x_c)| = |\sin(0.001) - 0.001|\approx1.6\times10^{-10}$

In [ ]:
x = 0.001
print(np.abs(np.sin(x)-x))

### Definition of Multiple Root

Assume that $r$ is a root of the differentiable function $f$; that is, assume that $f(r)= 0$. Then if $0=f(r)=f′(r)=f′′(r)=\cdots= f^{(m−1)}(r)$, but $f^{(m)}(r)\neq0$, we say that $f$ has a **root** of **multiplicity** $m$ at $r$. We say that $f$ has a **multiple root** at $r$ if the multiplicity is greater than one. The root is called **simple** if the multiplicity is one.